In [16]:
import os
import sys
import numpy as np
from skimage import io
import SimpleITK as sitk
from skimage.filters import gaussian

In [2]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data'
moving_path = os.path.join(DATA, 'MD585/preps/CH1/moving_volume.tif')
fixed_path = os.path.join(DATA, 'MD589/preps/CH1/moving_volume.tif')
print(fixed_path)

/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/MD589/preps/CH1/moving_volume.tif


In [45]:
def normalize8(img):
    mn = img.min()
    mx = img.max()
    mx -= mn
    img = ((img - mn)/mx) * 2**8 - 1
    return np.round(img).astype(np.uint8) 

def pad_volume(size, volume):
    size_difference = size - volume.shape
    xr, yr, zr = ((size_difference)/2).astype(int)
    xl, yl, zl = size_difference - np.array([xr, yr, zr])
    return np.pad(volume, [[xl, xr], [yl, yr], [zl, zr]])


In [3]:
fixed_image = sitk.ReadImage(fixed_path, sitk.sitkFloat32)
moving_image = sitk.ReadImage(moving_path, sitk.sitkFloat32)

In [4]:
iterationNumbers = 600
spatialSamples = 6000
simpleElastix = sitk.ElastixImageFilter()
simpleElastix.SetParameterMap(sitk.GetDefaultParameterMap('translation'))
simpleElastix.AddParameterMap(sitk.GetDefaultParameterMap('affine'))
simpleElastix.AddParameterMap(sitk.GetDefaultParameterMap('bspline'))
simpleElastix.SetParameter("Registration","MultiMetricMultiResolutionRegistration")
simpleElastix.SetParameter( "Metric", ("NormalizedMutualInformation", "CorrespondingPointsEuclideanDistanceMetric",))
simpleElastix.SetParameter("Metric0Weight", "0.0")
simpleElastix.SetParameter("MaximumNumberOfIterations" , str(iterationNumbers))
simpleElastix.SetParameter("NumberOfSpatialSamples" , str(spatialSamples))
#simpleElastix.PrintParameterMap()

<SimpleITK.SimpleITK.ElastixImageFilter; proxy of <Swig Object of type 'itk::simple::ElastixImageFilter::Self *' at 0x7fa36612cc00> >

In [5]:
%%time
simpleElastix.SetFixedImage(fixed_image)
simpleElastix.SetMovingImage(moving_image)
simpleElastix.SetOutputDirectory('/tmp')
simpleElastix.SetFixedPointSetFileName("/tmp/fixed_points.pts")
simpleElastix.SetMovingPointSetFileName("/tmp/moving_points.pts")
simpleElastix.LogToConsoleOff()
resultImage = simpleElastix.Execute()

CPU times: user 13min 3s, sys: 7.93 s, total: 13min 11s
Wall time: 9min 53s


In [6]:
img = sitk.GetArrayFromImage(resultImage)
type(img), img.dtype, img.shape

(numpy.ndarray, dtype('float32'), (447, 253, 342))

In [7]:
savepath = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/MD585/preps/CH1/registered_585.tif'
io.imsave(savepath, img)

In [64]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
MD585_volumepath = os.path.join(DATA, 'MD585_sagittal.tif')
MD589_volumepath = os.path.join(DATA, 'MD589_14um_sagittal.tif')
MD594_volumepath = os.path.join(DATA, 'MD594_sagittal.tif')

In [65]:
MD585_volume = io.imread(MD585_volumepath)
MD589_volume = io.imread(MD589_volumepath)
MD594_volume = io.imread(MD594_volumepath)
volumes = [MD585_volume, MD589_volume, MD594_volume]

In [66]:
MD585_volume.dtype

dtype('float32')

In [67]:
#margin = 0
#sizes = np.array([vi.shape for vi in volumes])
#volume_size = sizes.max(0) + margin
#volumes = [normalize8(v) for v in volumes]
#volumes = [pad_volume(volume_size, vi) for vi in volumes]
#volumes = list([(v > 0).astype(np.int32) for v in volumes])
#for v in volumes:
#    print(v.dtype, v.shape)

merged_volume = np.sum(volumes, axis=0)
merged_volume = merged_volume / float(np.max(merged_volume))
#merged_volume_prob = merged_volume / 3
# increasing the STD makes the volume smoother
# Smooth the probability
#average_volume = gaussian(merged_volume_prob, 1.0)
#color = 255
#average_volume[average_volume > 0.8] = color
#average_volume[average_volume != color] = 0
#average_volume = average_volume.astype(np.uint8)
arr = normalize8(merged_volume)
#ids, counts = np.unique(arr, return_counts=True)
#print(ids, counts)

In [68]:
arr.dtype, arr.shape

(dtype('uint8'), (447, 253, 342))

In [69]:
savepath = os.path.join(DATA, 'gauss.tif')
io.imsave(savepath, arr)